In [57]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window

spark = (SparkSession.builder
        .master("local[*]")
        .appName("workshop_spark")
        .getOrCreate())

municipios = spark.read.option("header", True).csv("../datalake/raw/municipios.csv")
populacao = spark.read.option("header", True).csv("../datalake/raw/populacao_municipios.csv")
nomes = spark.read.option("header", True).csv("../datalake/raw/nomes_municipios.csv")

In [58]:
#filtrar populacao para 2010
print(populacao.count())
populacao_f = populacao.where(col("ano") == 2010)
print(populacao_f.count())
print(municipios.count())
municipios_f = (municipios
                .select("id_municipio", "municipio", "capital_estado", "estado", "regiao"))
print(municipios_f.count())

nomes = nomes.withColumn("qtde_nascimentos_ate_2010", nomes["qtde_nascimentos_ate_2010"].cast(IntegerType()))

#Join
df = (nomes.join(municipios_f, "id_municipio", how="left")
           .join(populacao_f, "id_municipio", how="left"))

1000
37
5570
5570


In [59]:
df.dtypes

[('id_municipio', 'string'),
 ('nome', 'string'),
 ('qtde_nascimentos_ate_2010', 'int'),
 ('municipio', 'string'),
 ('capital_estado', 'string'),
 ('estado', 'string'),
 ('regiao', 'string'),
 ('ano', 'string'),
 ('populacao', 'string')]

In [60]:
(df.select("nome", "qtde_nascimentos_ate_2010")
  .groupby("nome")
  .agg( sum("qtde_nascimentos_ate_2010")
        .alias("qtde_nascimentos_ate_2010"))
  .sort(col("qtde_nascimentos_ate_2010").desc())
  .limit(5).toPandas())

,nome,qtde_nascimentos_ate_2010
0,Maria,385816
1,Jose,163729
2,Ana,99245
3,Antonio,94619
4,Francisco,93829


In [61]:
df.sort(col("qtde_nascimentos_ate_2010").asc()).limit(5).toPandas()

,id_municipio,nome,qtde_nascimentos_ate_2010,municipio,capital_estado,estado,regiao,ano,populacao
0,1100015,Luzia,46,Alta Floresta D'Oeste,0,Rondônia,Norte,2010,24392
1,1100015,Edson,46,Alta Floresta D'Oeste,0,Rondônia,Norte,2010,24392
2,1100015,Julia,46,Alta Floresta D'Oeste,0,Rondônia,Norte,2010,24392
3,1100015,Ademir,46,Alta Floresta D'Oeste,0,Rondônia,Norte,2010,24392
4,1100015,Marcio,46,Alta Floresta D'Oeste,0,Rondônia,Norte,2010,24392


In [62]:
(df.select("nome", "id_municipio", "municipio")
  .groupby("nome")
  .agg( count("id_municipio")
        .alias("count_id_municipio"))
  .where(col("count_id_municipio") == 1)
  .limit(5).toPandas())

,nome,count_id_municipio
0,Uilian,1
1,Abner,1
2,Diomar,1
3,Eliude,1
4,Francileide,1


In [63]:
df.where(col("nome")=="Abner").limit(5).toPandas()

,id_municipio,nome,qtde_nascimentos_ate_2010,municipio,capital_estado,estado,regiao,ano,populacao
0,1302603,Abner,97,Manaus,1,Amazonas,Norte,None,None


In [ ]:
#qual nome tem a maior variancia?

In [64]:
(populacao.withColumn("populacao_anterior",
                     lag(col("populacao")) 
                     .over(Window 
                           .partitionBy("id_municipio")
                           .orderBy("ano")
                          ) 
                     )
         .withColumn("media_populacao_5_anos",
                     avg(col("populacao"))
                     .over(Window
                           .partitionBy("id_municipio")
                           .rowsBetween(0,4)
                           .orderBy(desc("ano"))
                          )
                    )
         .sort("id_municipio",desc("ano"))
         .limit(5).toPandas())


,id_municipio,ano,populacao,populacao_anterior,media_populacao_5_anos
0,1100015,2019,22945,23167,24526.6
1,1100015,2018,23167,25437,25068.0
2,1100015,2017,25437,25506,25580.2
3,1100015,2016,25506,25578,25306.6
4,1100015,2015,25578,25652,25051.0


In [ ]:
def squared(s):
    return s + s
squared2 = spark.udf.register("squaredWithPython", squared, LongType())
df_sp.withColumn("square", squared2(col("Confirmed").cast(IntegerType()))).show()